## **<font color="red">Function Tools</font>**
- **FunctionTool**
- **LongRunningFunctionTool**
- **Agent-as-a-Tool**
- ***Architecture***
```
User
  ↓
MainAgent
  ↓ decides which tool
  ├── FunctionTool (capital)
  ├── Async Tool (report)
  └── MathAgent (Agent as Tool)
```

In [4]:
# ============================================================
# COMPLETE ADK DEMO (COMPATIBLE VERSION)
# 1. Function Tool
# 2. Long Running Tool
# 3. Agent as Tool (Manual Implementation)
# ============================================================

import os
import asyncio
from typing import Optional

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool
from google.genai import types
from config import config

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY
MODEL = "gemini-2.0-flash"

APP_NAME = "multi_feature_app"
USER_ID = "user_1"
SESSION_ID = "session_001"

# ============================================================
# 1️⃣ NORMAL FUNCTION TOOL
# ============================================================

def get_capital_city(country: str) -> str:
    capitals = {
        "india": "New Delhi",
        "france": "Paris",
        "germany": "Berlin",
        "canada": "Ottawa"
    }
    return capitals.get(country.lower(), "Capital not found")

capital_tool = FunctionTool(func=get_capital_city)

# ============================================================
# 2️⃣ LONG RUNNING FUNCTION TOOL
# ============================================================

async def generate_report(topic: str) -> str:
    print("⏳ Generating report... please wait...")
    await asyncio.sleep(5)
    return f"Report on '{topic}' completed successfully."

long_running_tool = FunctionTool(func=generate_report)

# ============================================================
# 3️⃣ AGENT AS A TOOL (MANUAL WRAP)
# ============================================================

# Create sub-agent
math_agent = LlmAgent(
    name="MathAgent",
    model=MODEL,
    instruction="You are a math expert. Solve mathematical expressions carefully.",
    description="Solves math problems."
)

# Create its own session service
math_session_service = InMemorySessionService()

async def solve_math(problem: str) -> str:
    """
    This function wraps the MathAgent.
    It runs the agent and returns final response.
    """

    session_id = "math_session"

    # Create session if not exists
    if not await math_session_service.get_session(
        app_name="math_app",
        user_id="math_user",
        session_id=session_id
    ):
        await math_session_service.create_session(
            app_name="math_app",
            user_id="math_user",
            session_id=session_id
        )

    runner = Runner(
        agent=math_agent,
        app_name="math_app",
        session_service=math_session_service
    )

    content = types.Content(
        role="user",
        parts=[types.Part(text=problem)]
    )

    events = runner.run_async(
        user_id="math_user",
        session_id=session_id,
        new_message=content
    )

    final_answer: Optional[str] = None

    async for event in events:
        if event.is_final_response():
            final_answer = event.content.parts[0].text

    return final_answer or "No response from math agent."

# Wrap this wrapper function as tool
math_agent_tool = FunctionTool(func=solve_math)

# ============================================================
# MAIN AGENT
# ============================================================

main_agent = LlmAgent(
    name="MainAgent",
    model=MODEL,
    instruction="""
    You are a smart assistant.

    You can:
    - Find capital cities
    - Generate detailed reports
    - Solve math problems

    Use tools whenever appropriate.
    """,
    tools=[
        capital_tool,
        long_running_tool,
        math_agent_tool
    ]
)

# ============================================================
# RUNNER
# ============================================================

async def run_chat():

    session_service = InMemorySessionService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=main_agent,
        app_name=APP_NAME,
        session_service=session_service
    )

    print("🤖 Multi-Feature ADK Demo Started")
    print("Type 'exit', 'quit' to quit.\n")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit", ""]:
            break

        content = types.Content(
            role="user",
            parts=[types.Part(text=user_input)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("Assistant:", event.content.parts[0].text)

# ============================================================
# ENTRY POINT
# ============================================================

if __name__ == "__main__":
    # asyncio.run(run_chat())
    await run_chat()


# --- Test ---
# # FunctionTool
# What is the capital of India?

# # LongTimeFunctionTool
# Generate a report about AI agents.

# # Agent-as-Tool
# What is 2345 * 987?

🤖 Multi-Feature ADK Demo Started
Type 'exit', 'quit' to quit.



You:  
